<h1 align="center">Construindo um assistente pessoal com LlamaIndex e GPT-3.5</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

# Contextualizando

Com o recente lançamento da `API` da série `GPT 3.5` da `OpenAI`, agora é possível criar seu próprio `chatbot de perguntas e respostas` com base em seus próprios dados.


Neste script, exploraremos como criar um chatbot de `Q&A` de documentos de forma eficiente com o `llama-index` e a `API GPT`.


Um aplicativo interessante é o `Question Answering (QA)`, que permite que o bot recupere informações de documentos e forneça respostas rápidas às suas consultas em linguagem natural. Podemos usar esse avançado sistema de `NLP` para diversos fins, `como simplificar o suporte ao cliente`, `sintetizar a pesquisa do usuário` e `gerenciar o conhecimento pessoal`. Leve sua produtividade para o próximo nível com a `API GPT 3.5!`


## <font color="red">A ideia</font>


A [ideia](https://medium.com/artificialis/building-personal-assistant-with-llamaindex-and-gpt-3-5-934a9ba1fd89) de usar o `ChatGPT` como um assistente para sintetizar o feedback do cliente ou encontrar documentos de produtos antigos relacionados a um recurso que está sendo trabalhado é um caso de uso em potencial. Inicialmente, o `fine-tune do modelo GPT` com dados específicos foi considerado para atingir esse objetivo, mas essa abordagem requer um `grande conjunto de dados` e é cara. O fine-tune do modelo pode apenas ensinar uma nova habilidade, em vez de fornecer informações completas sobre os documentos. Para QA de (vários) documentos, a engenharia de prompt fornecendo contexto nos prompts é outra abordagem. Por outro lado, o `modelo GPT` tem um capacidade de atenção limitada e passar um contexto longo para a `API` pode ser caro, visto que existem milhares de e-mails de feedback de clientes e centenas de documentos de produtos.

Enquanto pesquisava maneiras de superar as `limitações do limite de token de entrada do prompt`, tive a ideia de usar um algoritmo para pesquisar documentos e extrair apenas os trechos relevantes. Ao passar esses contextos relevantes junto com minhas perguntas para o modelo `GPT`, esperava alcançar melhores resultados. Durante minha pesquisa, descobri a biblioteca `gpt-index`, que mais tarde foi renomeada para `LlamaIndex`. Essa biblioteca forneceu uma solução direta para o meu problema e me permitiu implementar facilmente a ideia.



# Construindo um ChatBot de QA sobre Documentos

Usaremos `LlamaIndex` e `GPT` (`text-davinci-003`) para criar um `chatbot` de `Q&A` que opera em documentos existentes.

### Pré-requisitos

* Uma chave de `API OpenAI`, que pode ser obtida [AQUI](https://platform.openai.com/account/api-keys)

* Um banco de dados (`database`) de seus documentos. O `LlamaIndex` oferece suporte a várias fontes de dados, como `Notion` ou `Google Docs`. Para este tutorial, um arquivo de texto simples será usado para demonstração.

* Um ambiente `Python` local ou um notebook on-line do `Google Colab`.


### Etapas 

* Crie um índice dos dados do seu documento utilizando o `LlamaIndex`

* Formule uma consulta de linguagem natural para pesquisar o índice

* O `LlamaIndex` recuperará as partes pertinentes do documento e as fornecerá ao `prompt do GPT`

* Faça a pergunta ao GPT com o contexto relevante e gere uma resposta.

O `LlamaIndex` cria um `índice vetorizado` a partir dos dados do documento, tornando a consulta altamente eficiente. Em seguida, ele usa esse `índice` para identificar as seções mais relevantes do documento com base na semelhança entre a consulta e os dados. As informações recuperadas são incorporadas ao prompt enviado ao GPT, fornecendo o contexto necessário para responder à sua pergunta.

Vamos fazê-lo!

A seguir, esses comandos instalarão o `LlamaIndex` e o `OpenAI`.

In [ ]:
%pip install llama-index
%pip install openai

In [ ]:
%pip install python-dotenv

In [49]:
import os
import openai
from llama_index import GPTSimpleVectorIndex, Document, SimpleDirectoryReader



# Isto é quando usas o arquivo .env: 
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY = os.environ['OPENAI_API_KEY']  
openai.api_key = Eddy_API_KEY 



Carregando a minha chave Key:  True


In [50]:
import pandas as pd

csv = pd.read_csv('/home/eddygiusepe/1_Eddy_Giusepe/3_estudando_LLMs/Large_Language_Models_LLMs/resposta_text.csv', lineterminator='\n')
csv.head()


,resposta_text
0,agendamento
1,caso o cidadão selecione o setor correto mas ...
2,quais os canais de atendimento para esclarece...
3,orientações para inscrição creche qual a docum...
4,administrações regionais águas claras - ra x...


In [51]:
import pandas as pd

df = pd.read_csv('/home/eddygiusepe/1_Eddy_Giusepe/3_estudando_LLMs/Large_Language_Models_LLMs/pre-processed.csv')
df.head()

,index,label,preprocessed_news
0,0,fake,katia abreu diz vai colocar expulsao moldura n...
1,1,fake,ray peita bolsonaro conservador fake entrevist...
2,2,fake,reinaldo azevedo desmascarado policia federal ...
3,3,fake,relatorio assustador bndes mostra dinheiro pub...
4,4,fake,radialista americano fala sobre pt vendem ilus...


In [44]:
df.drop(['index', 'label'], axis = 1, inplace = True)

In [46]:
#df.to_csv('./data/fakenews.csv', index=False)

Agora precisamos construir um índice do nosso documento.

In [52]:
documents = SimpleDirectoryReader('/home/eddygiusepe/1_Eddy_Giusepe/3_estudando_LLMs/Large_Language_Models_LLMs/Building_personal_assistant_with_LlamaIndex_and_GPT-3.5/data').load_data()

# Descomentar o seguinte para poder carregar qualquer arquvio .csv: 

# from pathlib import Path
# from llama_index import download_loader

# SimpleCSVReader = download_loader("SimpleCSVReader")

# loader = SimpleCSVReader()
# documents = loader.load_data(file=Path('./data/fakenews.csv'))



<font color="orange">Depois de carregar os documentos, podemos então construir o índice:</font>

In [53]:
index = GPTSimpleVectorIndex(documents)

INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_documents] Total embedding token usage: 453 tokens


<font color="orange">Consultando o índice:</font>

In [55]:
resposta = index.query("Quando de dineiro devo ter se eu quiser comprar uma camisa?") 
print(resposta)


INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 281 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 21 tokens



R$ 30,00


# Conclusão:

Neste pequeno script demonstrou a eficácia da combinação de `GPT`com LlamaIndex para a criação de um `ChatBot` de resposta a perguntas de Documentos. 

Embora o `GPT` sozinho seja uma ferramenta impressionante, seus recursos podem ser bastante aprimorados pela integração com outras ferramentas, dados e processos.